<a href="https://colab.research.google.com/github/Pdugovich/DS-Unit-2-Kaggle-Challenge/blob/master/module4/assignment_kaggle_challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Assignment 4

## Previous Day's Code

Basically unchanged aside from reducing the number of dropped duplicate columns to 'payment_type', 'quantity_group', and 'extraction_type_group'. Of the "nearly-duplicate" columns, 'extraction_type_group' was the only one, when removed, that improved my val accuracy.

Also added the 'year_difference' column.

In [0]:
import sys

DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [0]:
#Checked the average elevation of Tanzania, and it's 1200, so I 
#really can ignore the zeroes here. But also the lowest elevation is 0,
#so I'm a little confused about the negative numbers
train['gps_height'].value_counts()

 0       20438
-15         60
-16         55
-13         55
-20         52
         ...  
 2285        1
 2424        1
 2552        1
 2413        1
 2385        1
Name: gps_height, Length: 2428, dtype: int64

In [0]:
# Numeric Columns to clean
numeric_to_clean = ['longitude','latitude','construction_year', 'gps_height']

In [0]:
# Checking for duplicat columns
duplicates1 = ['extraction_type','extraction_type_group','extraction_type_class']
duplicates2 = ['payment','payment_type']
duplicates3 = ['quantity_group','quantity']
duplicates4 = ['source','source_type']
duplicates5 = ['waterpoint_type','waterpoint_type_group']
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [0]:
#Checking the duplicates to decide which to keep
duplicate_lists = [duplicates1, duplicates2, duplicates3, duplicates4,
                   duplicates5]

for duplicate in duplicate_lists:
  print(train[duplicate].describe())
  print("")

       extraction_type extraction_type_group extraction_type_class
count            59400                 59400                 59400
unique              18                    13                     7
top            gravity               gravity               gravity
freq             26780                 26780                 26780

          payment payment_type
count       59400        59400
unique          7            7
top     never pay    never pay
freq        25348        25348

       quantity_group quantity
count           59400    59400
unique              5        5
top            enough   enough
freq            33186    33186

        source source_type
count    59400       59400
unique      10           7
top     spring      spring
freq     17021       17021

           waterpoint_type waterpoint_type_group
count                59400                 59400
unique                   7                     6
top     communal standpipe    communal standpipe
freq                

In [0]:
#my_train['region'].value_counts().index

In [0]:
Mwanza = train[train['region'] == 'Mwanza']
Mwanza['longitude'] = Mwanza['longitude'].replace(0,np.nan)
Mwanza['latitude'] = Mwanza['latitude'].replace(-2e-08, np.nan)

D:\Users\Patch\Anaconda3\envs\unit\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\Patch\Anaconda3\envs\unit\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
np.mean(Mwanza['latitude'])

-2.6205017775686277

In [0]:
np.mean(Mwanza['longitude'])

33.09156419778649

In [0]:
Shinyanga = train[train['region'] == 'Shinyanga']
Shinyanga['longitude'] = Shinyanga['longitude'].replace(0,np.nan)
Shinyanga['latitude'] = Shinyanga['latitude'].replace(-2e-08, np.nan)

D:\Users\Patch\Anaconda3\envs\unit\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\Patch\Anaconda3\envs\unit\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
np.mean(Shinyanga['longitude'])

33.24012071028917

In [0]:
np.mean(Shinyanga['latitude'])

-3.495696017133518

In [0]:
#Looking at the above lists, I'll remove the duplicate columns
# and nearly duplicate columns that have fewer unique variables

duplicates_to_drop = ['extraction_type_group','extraction_type_class',
                    'payment_type','quantity_group', 'source_type',
                    'waterpoint_type_group']

In [0]:
from sklearn.model_selection import train_test_split

my_train, my_val = train_test_split(train, random_state=333)

In [0]:
my_train[my_train['longitude']==0]['region'].value_counts()

Shinyanga    763
Mwanza       623
Name: region, dtype: int64

In [0]:
my_train[my_train['region']== 'Mwanza']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
3353,41817,0.0,2011-07-21,NaN,1183,NaN,33.111956,-2.125932e+00,Office,0,...,unknown,unknown,unknown,unknown,unknown,other,unknown,hand pump,hand pump,non functional
16640,64071,0.0,2011-08-08,Hesawa,1149,DWE,33.020814,-1.870797e+00,Church,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional
26365,24605,0.0,2011-07-18,Hesawa,0,DWE,33.251206,-2.536370e+00,Dyuya,0,...,soft,good,insufficient,insufficient,spring,spring,groundwater,other,other,non functional
51183,2206,0.0,2011-07-28,Government Of Tanzania,0,Government,0.000000,-2.000000e-08,Mahakama,0,...,unknown,unknown,dry,dry,dam,dam,surface,communal standpipe multiple,communal standpipe,non functional
52506,8245,0.0,2011-08-08,Hesawa,0,DWE,33.515992,-2.651120e+00,Mwabashola,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18963,41164,0.0,2011-07-25,Bkhws,0,BKHWS,32.919408,-2.711763e+00,Loundy,0,...,soft,good,enough,enough,lake,river/lake,surface,communal standpipe,communal standpipe,non functional
49091,60405,0.0,2011-07-28,Hesawa,0,HESAWA,32.980052,-2.658572e+00,Kwa Kilo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,functional
44103,28939,0.0,2011-07-30,Hesawa,0,DWE,33.048568,-2.460987e+00,Kwa Dundo,0,...,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other,non functional
41317,47225,0.0,2011-07-30,Unicef,0,Maswi,33.069096,-2.431699e+00,Kwa Matayo,0,...,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional


In [0]:
import numpy as np
def wrangle(X):
  
  # To prevent copy warnings
  X = X.copy()

  # Latitude is strange in that it doesn't have any 0s, but it does have these
  # near-0 values
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # there are numeric columns with 0s that should be nana
  nans_as_zeros = ['latitude','longitude', 'construction_year',
                   'gps_height', 'population']
  for column in nans_as_zeros:
    X[column] = X[column].replace(0, np.nan)
    #I like this code Ryan had to make a new column for if its missing data
    X[column+'_MISSING'] = X[column].isnull()    
  
  # X['longitude'] = X.apply(
  #   lambda row: np.mean(Mwanza['longitude']) if np.isnan(row['longitude']) and row['region'] == 'Mwanza' else row['longitude'],
  #   axis=1)
  # X['longitude'] = X.apply(
  #   lambda row: np.mean(Shinyanga['longitude']) if np.isnan(row['longitude']) and row['region'] == 'Shinyanga' else row['longitude'],
  #   axis=1)


  # X['latitude'] = X.apply(
  #   lambda row: np.mean(Mwanza['latitude']) if np.isnan(row['latitude']) and row['region'] == 'Mwanza' else row['latitude'],
  #   axis=1)
  # X['latitude'] = X.apply(
  #   lambda row: np.mean(Shinyanga['latitude']) if np.isnan(row['latitude']) and row['region'] == 'Shinyanga' else row['latitude'],
  #   axis=1)
  
  #Date recorded is treated as an int. Extracting y/m/d
  X['date_recorded'] = pd.to_datetime(X['date_recorded'])
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')
    
  # Engineer feature: how many years from construction_year to date_recorded
  X['years_difference'] = X['year_recorded'] - X['construction_year']    

  #Removing duplicate or near-duplicate columns
  X = X.drop(columns=['payment_type','quantity_group', 'extraction_type_group'])

  #Can be used for each train and validation
  return X

In [0]:
%%time
my_train = wrangle(my_train)
my_val = wrangle(my_val)
test = wrangle(test)

Wall time: 273 ms


In [0]:
my_train['longitude'].value_counts()

39.103752    2
39.096499    2
37.530515    2
37.339811    2
37.542785    2
            ..
31.701989    1
37.131667    1
34.607110    1
33.076362    1
35.005922    1
Name: longitude, Length: 43123, dtype: int64

In [0]:
my_train[my_train['id']== 6091]

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,status_group,latitude_MISSING,longitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,year_recorded,month_recorded,day_recorded,years_difference


In [0]:
# # Copied from previous assignment. 
# # Unecessary because high cardinality features are fine

# # # Selecting target

# target = 'status_group'

# #Removing the target and useless id columns
# train_columns = my_train.drop(columns=[target,'id'])

# # separating numeric columns to readd after
# numeric_columns = train_columns.select_dtypes(include='number').columns.tolist()

# #Getting a list of cardinality for categorical features to exclude the large
# cardinality = train_columns.select_dtypes(exclude='number').nunique()

# #Excluding features with a cardinality over 50
# categorical_columns = cardinality[cardinality <50].index.tolist()

# #combining lists to get the features I will use for my model
# features = numeric_columns + categorical_columns

In [0]:
# We can use high cardinality features, so no need to remove them
target = 'status_group'

features = my_train.drop(columns=[target,'id']).columns

In [0]:
#Assigning variables

X_train = my_train[features]
y_train = my_train[target]

X_val = my_val[features]
y_val = my_val[target]

X_test = test[features]

## Assignment 4 code

Went with the column transformer to use ordinal encoding ONLY for the high cardinality columns, one hot encoding otherwise.

In [0]:
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

#Getting a list of cardinality for categorical features to include in ordinal encoder
train_columns = my_train.drop(columns=[target,'id'])
cardinality = train_columns.select_dtypes(exclude='number').nunique()
high_cardinality = cardinality[cardinality >49].index.tolist()

# Making the high cardinality pipeline
high_cardinality_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median')
)

# Making the non-high cardinality pipeline
non_high_cardinality = train_columns.drop(columns=high_cardinality).columns

non_high_cardinality_pipeline = make_pipeline(
    ce.OneHotEncoder(),
    SimpleImputer(strategy='median')
)


#Associating the transformers with the list of features
preprocessor = ColumnTransformer(
    transformers=[
        ('high_card',high_cardinality_pipeline, high_cardinality),
        ('low_card',non_high_cardinality_pipeline, non_high_cardinality)])


my_pipeline = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_estimators=100, random_state=333, n_jobs=-1,
                           max_depth=20)
)

In [0]:
my_pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('high_card',
                                                  Pipeline(memory=None,
                                                           steps=[('ordinalencoder',
                                                                   OrdinalEncoder(cols=None,
                                                                                  drop_invariant=False,
                                                                                  handle_missing='value',
                                                                                  handle_unknown='value',
                                                                                  mapping=None,
                               

In [0]:
my_pipeline.score(X_val,y_val)

0.805993265993266

In [0]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


Set up the RandomizedSearchCV to 

In [0]:
%%time

param_distributions = {
    'randomforestclassifier__n_estimators': randint(100, 200),
    'randomforestclassifier__max_depth': [17, 18,19,20,21,22, None],
    'randomforestclassifier__min_samples_leaf': [1,2,3,4,5]
}

search = RandomizedSearchCV(
    my_pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=False,
    n_jobs=-1
)

Wall time: 1 ms


In [0]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  7.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('columntransformer',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('high_card',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('ordinalencoder',
                                                                                                OrdinalEncoder(cols=None,
                                                            

In [0]:
search.best_score_

0.8111111111111111

In [0]:
search.best_params_

{'randomforestclassifier__max_depth': 21,
 'randomforestclassifier__min_samples_leaf': 1,
 'randomforestclassifier__n_estimators': 194}

In [0]:
search.best_estimator_

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('high_card',
                                                  Pipeline(memory=None,
                                                           steps=[('ordinalencoder',
                                                                   OrdinalEncoder(cols=None,
                                                                                  drop_invariant=False,
                                                                                  handle_missing='value',
                                                                                  handle_unknown='value',
                                                                                  mapping=None,
                               

## Submission Code

In [0]:
pipeline = search.best_estimator_

In [0]:
pred_y_test = pipeline.predict(X_test)

In [0]:
cv8_submission = test.copy()

In [0]:
cv8_submission['status_group'] = pred_y_test

In [0]:
cv8_submission = cv8_submission.filter(['id','status_group'])

In [0]:
cv8_submission

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14353,39307,non functional
14354,18990,functional
14355,28749,functional
14356,33492,functional


In [0]:
cv8_submission.to_csv('cv8_submission.csv', index=False)

## The code below used to combine the my_train and my_val data to use in the final fitting with the randomCV optomized model

In [0]:
my_train.shape,my_val.shape

((44550, 46), (14850, 46))

In [0]:
# Combining all training data to use with the best model the randomcv found
final_train = pd.concat([my_train,my_val], ignore_index=True)

In [0]:
final_train.shape

(59400, 46)

In [0]:
final_train.head()

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,status_group,latitude_MISSING,longitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,year_recorded,month_recorded,day_recorded,years_difference
0,56413,0.0,Dhv,362.0,DWE,36.760514,-8.970829,Shuleni,0,Rufiji,...,functional,False,False,False,False,False,2011,3,4,17.0
1,33507,20.0,Tanapa,391.0,TA,38.006725,-6.080969,Kwa Mzee Omary,0,Wami / Ruvu,...,functional,False,False,False,False,False,2011,3,8,1.0
2,62973,25.0,Government Of Tanzania,855.0,Government,37.728087,-4.062194,Kwa Rabeka,0,Pangani,...,functional,False,False,False,False,False,2013,3,4,3.0
3,36171,0.0,Government Of Tanzania,1474.0,DWE,37.497065,-3.282689,Kwa Paterino Shayo,0,Pangani,...,non functional,False,False,False,False,False,2013,3,26,41.0
4,30341,0.0,Commu,2044.0,Commu,34.140166,-9.277889,none,0,Lake Nyasa,...,functional,False,False,False,False,True,2011,2,21,2.0


In [0]:
pipeline.fit(final_train[features],final_train[target])

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('high_card',
                                                  Pipeline(memory=None,
                                                           steps=[('ordinalencoder',
                                                                   OrdinalEncoder(cols=None,
                                                                                  drop_invariant=False,
                                                                                  handle_missing='value',
                                                                                  handle_unknown='value',
                                                                                  mapping=None,
                               